In [8]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from PIL import Image
import cv2
import torch
import yaml
import supervision as sv
from ultralytics import YOLO
from sklearn.model_selection import train_test_split
import shutil  # Import the shutil module



In [9]:

# Set seeds for reproducibility
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

# Define paths
base_path = "./"  # Update this to your actual path
dataset_path = os.path.join(base_path, "Dataset")
labels_path = os.path.join(base_path, "Labels")

# Prepare folders for YOLO format
os.makedirs("solar_panels/images/train", exist_ok=True)
os.makedirs("solar_panels/images/val", exist_ok=True)
os.makedirs("solar_panels/images/test", exist_ok=True)
os.makedirs("solar_panels/labels/train", exist_ok=True)
os.makedirs("solar_panels/labels/val", exist_ok=True)
os.makedirs("solar_panels/labels/test", exist_ok=True)



In [10]:

# Collect data
def collect_data():
    data = []

    # Processing HD resized images
    hd_images_path = os.path.join(dataset_path, "hd_resized")
    hd_labels_path = os.path.join(labels_path, "labels_hd")

    for img_file in os.listdir(hd_images_path):
        if img_file.endswith(('.jpg', '.jpeg', '.png')):
            img_path = os.path.join(hd_images_path, img_file)
            # Assuming label files have same name but .txt extension
            label_file = os.path.splitext(img_file)[0] + '.txt'
            label_path = os.path.join(hd_labels_path, label_file)

            if os.path.exists(label_path):
                data.append((img_path, label_path))

    # Process native resized images
    native_images_path = os.path.join(dataset_path, "native_resized")
    native_labels_path = os.path.join(labels_path, "native_hd")

    for img_file in os.listdir(native_images_path):
        if img_file.endswith(('.jpg', '.jpeg', '.png')):
            img_path = os.path.join(native_images_path, img_file)
            # Assuming label files have same name but .txt extension
            label_file = os.path.splitext(img_file)[0] + '.txt'
            label_path = os.path.join(native_labels_path, label_file)

            if os.path.exists(label_path):
                data.append((img_path, label_path))

    return data


# Split data into train, validation, and test sets
def split_data(data):
    # 80-20 split for train-test
    train_data, test_data = train_test_split(
        data, test_size=0.2, random_state=42)

    # 10% of training data for validation (which is 0.1 * 0.8 = 0.08 of total data)
    train_data, val_data = train_test_split(
        train_data, test_size=0.1, random_state=42)

    print(f"Train set: {len(train_data)} samples")
    print(f"Validation set: {len(val_data)} samples")
    print(f"Test set: {len(test_data)} samples")

    return train_data, val_data, test_data


# Prepare data in YOLO format
def prepare_yolo_data(train_data, val_data, test_data):
    # Use shutil.copy instead of os.symlink
    def copy_data(src, dst):
        try:
            shutil.copy(src, dst)  # Use shutil.copy
            print(f"Copied {src} to {dst}")
        except Exception as e:
            print(f"Error copying {src} to {dst}: {e}")
    # Process each split
    for idx, (img_path, label_path) in enumerate(train_data):
        # Copy or symlink files to YOLO structure
        img_filename = os.path.basename(img_path)
        label_filename = os.path.basename(label_path)

        # Copy files to YOLO structure
        copy_data(img_path, f"solar_panels/images/train/{img_filename}")
        copy_data(label_path, f"solar_panels/labels/train/{label_filename}")

    for idx, (img_path, label_path) in enumerate(val_data):
        img_filename = os.path.basename(img_path)
        label_filename = os.path.basename(label_path)

        copy_data(img_path, f"solar_panels/images/val/{img_filename}")
        copy_data(label_path, f"solar_panels/labels/val/{label_filename}")

    for idx, (img_path, label_path) in enumerate(test_data):
        img_filename = os.path.basename(img_path)
        label_filename = os.path.basename(label_path)

        copy_data(img_path, f"solar_panels/images/test/{img_filename}")
        copy_data(label_path, f"solar_panels/labels/test/{label_filename}")

    # Creating YAML file for YOLO
    yaml_content = {
        'path': os.path.abspath('solar_panels'),
        'train': 'images/train',
        'val': 'images/val',
        'test': 'images/test',
        'names': {
            0: 'solar_panel',
            1: 'other_panel' 
        }
    }

    with open('solar_panels/dataset.yaml', 'w') as f:
        yaml.dump(yaml_content, f)

    return 'solar_panels/dataset.yaml'



In [11]:

# Train YOLO model
def train_yolo_model(yaml_path):
    # Use a smaller model like YOLOv8n for faster training on Intel Iris X
    model = YOLO('yolov8n.pt')

    if torch.cuda.is_available() and "Iris" in torch.cuda.get_device_name(0):
        batch_size = 8
    else:
        batch_size = 16

    # Training the model
    results = model.train(
        data=yaml_path,
        epochs=50,
        patience=10,  # Early stopping
        batch=batch_size,
        imgsz=640,
        plots=True,  # Saving plots
        save=True,  # Saving model
    )

    return model, results



In [12]:

# Visualizign ground truth and predictions
def visualize_predictions(model, test_data):
    # Selecting 4 random samples
    samples = random.sample(test_data, min(4, len(test_data)))

    fig, axs = plt.subplots(2, 2, figsize=(15, 10))
    axs = axs.flatten()

    for i, (img_path, label_path) in enumerate(samples):
        # Loading image
        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        height, width = img.shape[:2]

        # Loading ground truth
        gt_boxes = []
        gt_classes = []
        with open(label_path, 'r') as f:
            for line in f:
                parts = line.strip().split(' ')
                class_id = int(parts[0])
                x_center, y_center = float(parts[1]), float(parts[2])
                box_width, box_height = float(parts[3]), float(parts[4])

                # Convert to pixel coordinates (left, top, width, height)
                x1 = int((x_center - box_width / 2) * width)
                y1 = int((y_center - box_height / 2) * height)
                w = int(box_width * width)
                h = int(box_height * height)

                gt_boxes.append([x1, y1, w, h])
                gt_classes.append(class_id)

        # Getting predictions
        results = model(img_path)
        pred_boxes = []
        pred_scores = []
        pred_classes = []

        for r in results:
            boxes = r.boxes
            for box in boxes:
                # Get xyxy format
                x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                # Convert to xywh format
                w, h = x2 - x1, y2 - y1
                pred_boxes.append([int(x1), int(y1), int(w), int(h)])
                pred_scores.append(float(box.conf))
                pred_classes.append(int(box.cls))

        # Display image
        axs[i].imshow(img)

        # Draw ground truth boxes (green)
        for j, (box, class_id) in enumerate(zip(gt_boxes, gt_classes)):
            x, y, w, h = box
            rect = patches.Rectangle(
                (x, y), w, h, linewidth=2, edgecolor='g', facecolor='none')
            axs[i].add_patch(rect)
            axs[i].text(x, y-5, f'GT: {class_id}',
                        color='g', fontsize=8, backgroundcolor='w')

        # Draw predicted boxes (red)
        for j, (box, class_id, score) in enumerate(zip(pred_boxes, pred_classes, pred_scores)):
            x, y, w, h = box
            rect = patches.Rectangle(
                (x, y), w, h, linewidth=2, edgecolor='r', facecolor='none')
            axs[i].add_patch(rect)
            axs[i].text(x, y+h+15, f'Pred: {class_id} ({score:.2f})',
                        color='r', fontsize=8, backgroundcolor='w')

        axs[i].set_title(f"Sample {i+1}")
        axs[i].axis('off')

    plt.tight_layout()
    plt.savefig('prediction_comparison.png')
    plt.show()

    return samples



In [13]:
#Computing metrics
iou_thresholds = [0.1, 0.3, 0.5, 0.7, 0.9]
# Confidence thresholds
conf_thresholds = [0.1, 0.3, 0.5, 0.7, 0.9]

# For mAP calculation
all_ground_truths = []
all_predictions = []


def metrics(model, test_data):
    # Processing all test samples
    global all_ground_truths, all_predictions
    all_ground_truths = []
    all_predictions = []

    # Lists to store individual detections
    gt_xyxy = []
    gt_confidence = []
    gt_class_id = []

    pred_xyxy = []
    pred_confidence = []
    pred_class_id = []

    for img_path, label_path in test_data:
        # Loadign image
        img = cv2.imread(img_path)
        height, width = img.shape[:2]

        # Parsing ground truth
        with open(label_path, 'r') as f:
            for line in f:
                parts = line.strip().split(' ')
                class_id = int(parts[0])
                x_center, y_center = float(parts[1]), float(parts[2])
                box_width, box_height = float(parts[3]), float(parts[4])

                # Converting to xyxy format
                x1 = (x_center - box_width / 2) * width
                y1 = (y_center - box_height / 2) * height
                x2 = (x_center + box_width / 2) * width
                y2 = (y_center + box_height / 2) * height

                gt_xyxy.append([x1, y1, x2, y2])
                gt_confidence.append(1.0)
                gt_class_id.append(class_id)

        # Getting predictions
        results = model(img_path)

        for r in results:
            boxes = r.boxes
            for box in boxes:
                x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                class_id = int(box.cls[0])
                confidence = float(box.conf[0])

                pred_xyxy.append([x1, y1, x2, y2])
                pred_confidence.append(confidence)
                pred_class_id.append(class_id)

    # Converting to numpy arrays
    gt_xyxy = np.array(gt_xyxy)
    gt_confidence = np.array(gt_confidence)
    gt_class_id = np.array(gt_class_id)

    pred_xyxy = np.array(pred_xyxy)
    pred_confidence = np.array(pred_confidence)
    pred_class_id = np.array(pred_class_id)

    # Create sv.Detections objects
    ground_truth_detections = sv.Detections(
        xyxy=gt_xyxy,
        confidence=gt_confidence,
        class_id=gt_class_id
    )

    prediction_detections = sv.Detections(
        xyxy=pred_xyxy,
        confidence=pred_confidence,
        class_id=pred_class_id
    )

    # Calculate mAP50
    metric = sv.metrics.MeanAveragePrecision()
    metric.update(
        ground_truth_detections,
        prediction_detections
    )
    map_result = metric.compute()
    
    # Debug the map_result structure
    print(f"mAP result type: {type(map_result)}")
    print(f"mAP result attributes: {dir(map_result)}")
    print(f"Full mAP result: {map_result}")
    
    # Try different ways to access map50 value
    try:
        # Try attribute access
        if hasattr(map_result, "map50"):
            print(f"mAP50 using supervision (attribute): {map_result.map50}")
        # Try dictionary-like access with get method
        elif hasattr(map_result, "get") and callable(map_result.get):
            print(f"mAP50 using supervision (get method): {map_result.get('map50')}")
        # Try direct access if it's a dictionary
        elif isinstance(map_result, dict) and "map50" in map_result:
            print(f"mAP50 using supervision (dict access): {map_result['map50']}")
        else:
            print("Could not find map50 in the result. Please check the structure of the result object.")
    except Exception as e:
        print(f"Error accessing map50: {e}")

    # Manually implement mAP50
    def calculate_map50(gt_detections, pred_detections):
        # Simple mAP50 implementation for comparison
        # This is a simplified version and not as robust as the supervision one

        # Count total ground truth boxes
        total_gt = len(gt_detections.xyxy)
        if total_gt == 0:
            return 0.0

        # Count correct predictions (TP)
        tp = 0
        gt_used = [False] * len(gt_detections.xyxy)

        for i in range(len(pred_detections.xyxy)):
            if pred_detections.confidence[i] < 0.5:  # Only consider predictions with confidence >= 0.5
                continue

            # Find best matching ground truth
            best_iou = 0
            best_gt_idx = -1

            for j in range(len(gt_detections.xyxy)):
                if gt_used[j]:  # skip already used gt
                    continue

                # Calculate IoU
                xi1 = max(pred_detections.xyxy[i][0], gt_detections.xyxy[j][0])
                yi1 = max(pred_detections.xyxy[i][1], gt_detections.xyxy[j][1])
                xi2 = min(pred_detections.xyxy[i][2], gt_detections.xyxy[j][2])
                yi2 = min(pred_detections.xyxy[i][3], gt_detections.xyxy[j][3])

                # Calculate intersection area
                inter_width = max(0, xi2 - xi1)
                inter_height = max(0, yi2 - yi1)
                inter_area = inter_width * inter_height

                # Calculate union area
                pred_area = (pred_detections.xyxy[i][2] - pred_detections.xyxy[i][0]) * (pred_detections.xyxy[i][3] - pred_detections.xyxy[i][1])
                gt_area = (gt_detections.xyxy[j][2] - gt_detections.xyxy[j][0]) * (gt_detections.xyxy[j][3] - gt_detections.xyxy[j][1])
                union_area = pred_area + gt_area - inter_area

                # Calculate IoU
                iou = inter_area / union_area if union_area > 0 else 0

                if iou > best_iou and iou >= 0.5 and pred_detections.class_id[i] == gt_detections.class_id[j]:
                    best_iou = iou
                    best_gt_idx = j

            if best_gt_idx >= 0:
                tp += 1
                gt_used[best_gt_idx] = True  # mark gt as used

        # Calculate precision (TP / (TP + FP))
        total_pred = len(pred_detections.xyxy)
        precision = tp / total_pred if total_pred > 0 else 0

        # Calculate recall (TP / Total GT)
        recall = tp / total_gt if total_gt > 0 else 0

        # For a basic mAP, we can use the average of precision and recall
        manual_map50 = (precision + recall) / 2

        return manual_map50

    my_map50 = calculate_map50(ground_truth_detections, prediction_detections)
    print(f"My mAP50 implementation: {my_map50}")

    # Create Precision, Recall, F1 table for different thresholds
    result_table = []
    header = ["IoU / Conf"] + [f"Conf={conf:.1f}" for conf in conf_thresholds]
    result_table.append(header)

    for iou_thresh in iou_thresholds:
        row = [f"IoU={iou_thresh:.1f}"]

        for conf_thresh in conf_thresholds:
            # Calculate TP, FP, FN manually
            tp, fp, fn = 0, 0, 0
            
            # Count true positives, false positives, false negatives manually
            gt_used = [False] * len(ground_truth_detections.xyxy)
            
            for i in range(len(prediction_detections.xyxy)):
                if prediction_detections.confidence[i] < conf_thresh:
                    continue
                    
                best_iou = 0
                best_gt_idx = -1
                
                for j in range(len(ground_truth_detections.xyxy)):
                    if gt_used[j]:
                        continue
                        
                    # Calculate IoU
                    xi1 = max(prediction_detections.xyxy[i][0], ground_truth_detections.xyxy[j][0])
                    yi1 = max(prediction_detections.xyxy[i][1], ground_truth_detections.xyxy[j][1])
                    xi2 = min(prediction_detections.xyxy[i][2], ground_truth_detections.xyxy[j][2])
                    yi2 = min(prediction_detections.xyxy[i][3], ground_truth_detections.xyxy[j][3])
                    
                    inter_width = max(0, xi2 - xi1)
                    inter_height = max(0, yi2 - yi1)
                    inter_area = inter_width * inter_height
                    
                    pred_area = (prediction_detections.xyxy[i][2] - prediction_detections.xyxy[i][0]) * \
                                (prediction_detections.xyxy[i][3] - prediction_detections.xyxy[i][1])
                    gt_area = (ground_truth_detections.xyxy[j][2] - ground_truth_detections.xyxy[j][0]) * \
                                (ground_truth_detections.xyxy[j][3] - ground_truth_detections.xyxy[j][1])
                    union_area = pred_area + gt_area - inter_area
                    
                    iou = inter_area / union_area if union_area > 0 else 0
                    
                    if iou > best_iou and iou >= iou_thresh and prediction_detections.class_id[i] == ground_truth_detections.class_id[j]:
                        best_iou = iou
                        best_gt_idx = j
                        
                if best_gt_idx >= 0:
                    tp += 1
                    gt_used[best_gt_idx] = True
                else:
                    fp += 1
                    
            fn = len(ground_truth_detections.xyxy) - sum(gt_used)

            # Calculate metrics
            precision = tp / (tp + fp) if (tp + fp) > 0 else 0
            recall = tp / (tp + fn) if (tp + fn) > 0 else 0
            f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0

            # Format as P/R/F1
            row.append(f"{precision:.2f}/{recall:.2f}/{f1:.2f}")

        result_table.append(row)

    # Printing as a formatted table
    for row in result_table:
        print(' | '.join(str(x) for x in row))


In [ ]:

if __name__ == "__main__":
    data = collect_data()

    # Splitting data
    train_data, val_data, test_data = split_data(data)

    # Prepare YOLO data
    yaml_path = prepare_yolo_data(train_data, val_data, test_data)

    # Train YOLO model
    model, results = train_yolo_model(yaml_path)
    
    # Now you can run your visualization and metrics functions
    samples = visualize_predictions(model, test_data)
    metrics(model, test_data)

Train set: 1829 samples
Validation set: 204 samples
Test set: 509 samples
Copied ./Dataset\hd_resized\solarpanels_hd_3__x0_21960_y0_20033_dxdy_832.jpg to solar_panels/images/train/solarpanels_hd_3__x0_21960_y0_20033_dxdy_832.jpg
Copied ./Labels\labels_hd\solarpanels_hd_3__x0_21960_y0_20033_dxdy_832.txt to solar_panels/labels/train/solarpanels_hd_3__x0_21960_y0_20033_dxdy_832.txt
Copied ./Dataset\hd_resized\solarpanels_hd_2__x0_4853_y0_8942_dxdy_832.jpg to solar_panels/images/train/solarpanels_hd_2__x0_4853_y0_8942_dxdy_832.jpg
Copied ./Labels\labels_hd\solarpanels_hd_2__x0_4853_y0_8942_dxdy_832.txt to solar_panels/labels/train/solarpanels_hd_2__x0_4853_y0_8942_dxdy_832.txt
Copied ./Dataset\hd_resized\solarpanels_hd_2__x0_5855_y0_10319_dxdy_832.jpg to solar_panels/images/train/solarpanels_hd_2__x0_5855_y0_10319_dxdy_832.jpg
Copied ./Labels\labels_hd\solarpanels_hd_2__x0_5855_y0_10319_dxdy_832.txt to solar_panels/labels/train/solarpanels_hd_2__x0_5855_y0_10319_dxdy_832.txt
Copied ./Datas

train: Scanning D:\Solar_Panel_Detection\solar_panels\labels\train.cache... 1829 images, 0 backgrounds, 115 corrupt: 100%|██████████| 1829/1829 [00:00<?, ?it/s]

train: WARNING  D:\Solar_Panel_Detection\solar_panels\images\train\solarpanels_hd_1__x0_0_y0_14041_dxdy_832.jpg: ignoring corrupt image/label: Label class 2 exceeds dataset class count 2. Possible class labels are 0-1
train: WARNING  D:\Solar_Panel_Detection\solar_panels\images\train\solarpanels_hd_1__x0_0_y0_14446_dxdy_832.jpg: ignoring corrupt image/label: Label class 2 exceeds dataset class count 2. Possible class labels are 0-1
train: WARNING  D:\Solar_Panel_Detection\solar_panels\images\train\solarpanels_hd_1__x0_1029_y0_14545_dxdy_832.jpg: ignoring corrupt image/label: Label class 2 exceeds dataset class count 2. Possible class labels are 0-1
train: WARNING  D:\Solar_Panel_Detection\solar_panels\images\train\solarpanels_hd_1__x0_1093_y0_15060_dxdy_832.jpg: ignoring corrupt image/label: Label class 2 exceeds dataset class count 2. Possible class labels are 0-1
train: WARNING  D:\Solar_Panel_Detection\solar_panels\images\train\solarpanels_hd_1__x0_14458_y0_19319_dxdy_832.jpg: ignor


val: Scanning D:\Solar_Panel_Detection\solar_panels\labels\val.cache... 204 images, 0 backgrounds, 9 corrupt: 100%|██████████| 204/204 [00:00<?, ?it/s]

val: WARNING  D:\Solar_Panel_Detection\solar_panels\images\val\solarpanels_hd_1__x0_0_y0_14020_dxdy_832.jpg: ignoring corrupt image/label: Label class 2 exceeds dataset class count 2. Possible class labels are 0-1
val: WARNING  D:\Solar_Panel_Detection\solar_panels\images\val\solarpanels_hd_1__x0_0_y0_14228_dxdy_832.jpg: ignoring corrupt image/label: Label class 2 exceeds dataset class count 2. Possible class labels are 0-1
val: WARNING  D:\Solar_Panel_Detection\solar_panels\images\val\solarpanels_hd_1__x0_122_y0_14455_dxdy_832.jpg: ignoring corrupt image/label: Label class 2 exceeds dataset class count 2. Possible class labels are 0-1
val: WARNING  D:\Solar_Panel_Detection\solar_panels\images\val\solarpanels_hd_1__x0_16433_y0_17271_dxdy_832.jpg: ignoring corrupt image/label: Label class 2 exceeds dataset class count 2. Possible class labels are 0-1
val: WARNING  D:\Solar_Panel_Detection\solar_panels\images\val\solarpanels_hd_1__x0_16489_y0_15532_dxdy_832.jpg: ignoring corrupt image/la

...


50 epochs completed in 7.204 hours.
Optimizer stripped from d:\Solar_Panel_Detection\runs\detect\train5\weights\last.pt, 6.2MB
Optimizer stripped from d:\Solar_Panel_Detection\runs\detect\train5\weights\best.pt, 6.2MB

Validating d:\Solar_Panel_Detection\runs\detect\train5\weights\best.pt...
Ultralytics 8.3.80  Python-3.10.16 torch-2.6.0+cpu CPU (13th Gen Intel Core(TM) i5-1335U)
Model summary (fused): 72 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:20<00:00,  2.96s/it]


                   all        195       2576      0.963       0.69      0.753      0.622
           solar_panel        195       2569      0.925      0.963       0.98      0.823
           other_panel          5          7          1      0.417      0.525       0.42
Speed: 1.8ms preprocess, 94.3ms inference, 0.0ms loss, 0.4ms postprocess per image
Results saved to d:\Solar_Panel_Detection\runs\detect\train5

image 1/1 d:\Solar_Panel_Detection\Dataset\hd_resized\solarpanels_hd_1__x0_14108_y0_19560_dxdy_832.jpg: 640x640 15 solar_panels, 187.2ms
Speed: 3.9ms preprocess, 187.2ms inference, 2.9ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 d:\Solar_Panel_Detection\Dataset\hd_resized\solarpanels_hd_3__x0_11885_y0_17740_dxdy_832.jpg: 640x640 3 solar_panels, 142.9ms
Speed: 4.1ms preprocess, 142.9ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 d:\Solar_Panel_Detection\Dataset\hd_resized\solarpanels_hd_3__x0_16230_y0_28423_dxdy_832.jpg: 640x640 6 sol

<Figure size 1500x1000 with 4 Axes>


image 1/1 d:\Solar_Panel_Detection\Dataset\hd_resized\solarpanels_hd_3__x0_10292_y0_26113_dxdy_832.jpg: 640x640 15 solar_panels, 146.5ms
Speed: 4.2ms preprocess, 146.5ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 d:\Solar_Panel_Detection\Dataset\hd_resized\solarpanels_hd_1__x0_3917_y0_15548_dxdy_832.jpg: 640x640 2 solar_panels, 145.0ms
Speed: 5.4ms preprocess, 145.0ms inference, 1.6ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 d:\Solar_Panel_Detection\Dataset\hd_resized\solarpanels_hd_1__x0_19021_y0_23183_dxdy_832.jpg: 640x640 6 solar_panels, 153.4ms
Speed: 4.7ms preprocess, 153.4ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 d:\Solar_Panel_Detection\Dataset\hd_resized\solarpanels_hd_2__x0_3779_y0_20194_dxdy_832.jpg: 640x640 4 solar_panels, 119.2ms
Speed: 4.6ms preprocess, 119.2ms inference, 0.9ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 d:\Solar_Panel_Detection\Dataset\hd_resized\solarp